Die Daten werden in **binärer Form** gesendet. Das bedeutet, dass die zu übertragenden Informationen als eine Folge von Bits (0 und 1) über das LoRa-Modul übertragen werden. Lass uns genauer betrachten, wie dieser Prozess funktioniert:

---

### **1. Datenrepräsentation: Von Bytes zu Binär**
- Im Python-Code werden die Daten typischerweise als **`bytearray`** oder **`bytes`** verarbeitet.
- Ein `bytearray` ist eine geordnete Sammlung von Bytes, wobei ein Byte 8 Bits entspricht.
- Beispiel:
  ```python
  data = b"Hello"
  ```
  - Die Zeichenkette `"Hello"` wird in Bytes konvertiert: `b'Hello'`.
  - Jedes Zeichen wird in seinem ASCII-Wert repräsentiert:
    - `H -> 72 (01001000)`
    - `e -> 101 (01100101)`
    - `l -> 108 (01101100)`
    - `o -> 111 (01101111)`

---

### **2. Aufbau der gesendeten Daten**
Die gesendeten Daten bestehen typischerweise aus zwei Teilen:
1. **Header**: 4 Bytes für Kontrollinformationen:
   - Zieladresse (`destination`)
   - Absenderadresse (`node`)
   - Sequenznummer (`identifier`)
   - Flags (`flags`)
2. **Payload (Nutzdaten)**:
   - Der eigentliche Inhalt, also die Nachricht, die du übermitteln möchtest.
   - Beispiel: `b"Hello"`

Zusammengeführt ergibt sich:
```python
payload = header + data
```

---

### **3. Übertragungsschritte**
Die Datenübertragung läuft folgendermaßen ab:

1. **FIFO-Schreiben**:
   - Die Daten (Header + Payload) werden in den FIFO-Speicher des LoRa-Moduls geschrieben.
   - Dies geschieht durch die Methode `_write_from`, die die Daten per SPI (Serial Peripheral Interface) an das Modul überträgt:
     ```python
     self._write_from(_RH_RF95_REG_00_FIFO, payload)
     ```

2. **Modulation (LoRa-Modul intern)**:
   - Das LoRa-Modul moduliert die binären Daten mit dem **Chirp Spread Spectrum (CSS)**-Verfahren:
     - Jeder Bitstrom wird durch spezielle Chirp-Signale dargestellt.
     - Diese Signale sind robust gegen Interferenzen und ermöglichen eine hohe Reichweite.

3. **Frequenzband**:
   - Die modulierten Daten werden auf das spezifizierte Frequenzband (z. B. 433 MHz oder 915 MHz) gesendet.
   - Dies geschieht auf der physikalischen Ebene, wobei das Signal über Antennen übertragen wird.

---

### **4. Empfang der Daten**
Beim Empfänger geschieht der umgekehrte Prozess:
1. **Demodulation**:
   - Das LoRa-Modul empfängt die modulierten Signale und demoduliert sie in binäre Daten.
2. **FIFO-Lesen**:
   - Die binären Daten werden aus dem FIFO-Speicher des Empfängers ausgelesen:
     ```python
     self._read_into(_RH_RF95_REG_00_FIFO, packet)
     ```
3. **Dateninterpretation**:
   - Die ersten 4 Bytes werden als Header interpretiert (z. B. Zieladresse).
   - Der Rest wird als Nutzdaten behandelt.

---

### **5. Beispiel eines binären Pakets**
Angenommen, du sendest:
- Zieladresse: `0x01`
- Absenderadresse: `0x02`
- Sequenznummer: `0x03`
- Flags: `0x04`
- Nutzdaten: `b'Hello'`

Das binäre Paket sieht folgendermaßen aus:
```text
Header:    00000001  00000010  00000011  00000100
Payload:   01001000  01100101  01101100  01101100  01101111
Gesamt:    00000001 00000010 00000011 00000100 01001000 01100101 01101100 01101100 01101111
```

---

### **6. Warum binär?**
Die Übertragung in binärer Form hat folgende Vorteile:
- **Effizienz**: Binäre Daten sind kompakt und einfach zu übertragen.
- **Robustheit**: Modulationstechniken wie LoRa nutzen binäre Signale, um Störungen zu minimieren.
- **Standardisierung**: Elektronische Geräte wie Mikrocontroller arbeiten nativ mit binären Daten.

---

### **Zusammenfassung**
- Die Daten werden binär übertragen.
- Im Code werden sie typischerweise als `bytearray` oder `bytes` verarbeitet.
- Das LoRa-Modul moduliert die binären Daten in ein robustes Funksignal (Chirp Spread Spectrum).
- Beim Empfänger wird das Signal demoduliert und wieder in binäre Daten zurückverwandelt.